In [35]:
#2-D Rastrigrin function
import random
import math
from random import Random
import copy

An = 10
minLimit = -5.12
maxLimit = 5.12
vector_length = 2
x_vec = [random.uniform(minLimit,maxLimit) for _ in range(vector_length)]

#Generate vectors
print(x_vec)

[-3.2131630210541156, -0.21751364593502576]


In [36]:
sum = 0
for i in range(vector_length):
    sum += x_vec[i]**2 - An*math.cos(2*math.pi*x_vec[i])

fitness = An + sum
print(fitness)

16.050769730299898


In [37]:
#Individuals(vector of floats)
#
# Individual.py
#
#

import math

#A simple 1-D Individual class
class Individual:
    """
    Individual
    """
    minSigma=1e-100
    maxSigma=1
    learningRate=1
    minLimit=None
    maxLimit=None
    uniprng=None
    normprng=None
    fitFunc=None
    vectorLength = None
    SearchMode         = "modeFindMIN"
    
    def __init__(self):
        self.vector_x = [self.uniprng.uniform(minLimit,maxLimit) for _ in range(self.vectorLength)]
        self.fit=self.__class__.fitFunc(self.vector_x)
        self.sigma=self.uniprng.uniform(0.9,0.1) #use "normalized" sigma
        
    def crossover(self, other):
        #perform crossover "in-place"
        alpha=self.uniprng.random()
        
        
        tmp=self.x*alpha+other.x*(1-alpha)
        other.x=self.x*(1-alpha)+other.x*alpha
        self.x=tmp
        
        self.fit=None
        other.fit=None
    
    def mutate(self):
        self.sigma=self.sigma*math.exp(self.learningRate*self.normprng.normalvariate(0,1))
        if self.sigma < self.minSigma: self.sigma=self.minSigma
        if self.sigma > self.maxSigma: self.sigma=self.maxSigma

        self.x=self.x+(self.maxLimit-self.minLimit)*self.sigma*self.normprng.normalvariate(0,1)
        self.fit=None
    
    def evaluateFitness(self):
        self.fit=self.__class__.fitFunc(self.vector_x)
        
    def __str__(self):
        print("--------------------------\n[INFO]Float Vector")
        print(f"x_vec:\n{self.vector_x}")
        print(f"Fit:\n{self.fit}")
        return '\n'



# Fitness Function N-dimensional Rastrigin

In [38]:
def fitnessFunc(x_vec):
    sum = 0
    for i in range(len(x_vec)):
        sum += x_vec[i]**2 - An*math.cos(2*math.pi*x_vec[i])

    fitness = An + sum
    return fitness

## Plotting out the 2-D Fitness function for basic visualization

# Instantiation

In [39]:
#start random number generators

randomSeed = 1234
uniprng=Random()
uniprng.seed(randomSeed)
normprng=Random()
normprng.seed(randomSeed+101)

#Set parameters
Individual.minSigma=1e-100
Individual.maxSigma=1
Individual.learningRate=1
Individual.minLimit=-5.12
Individual.maxLimit=5.12
Individual.uniprng= uniprng
Individual.normprng= normprng
Individual.fitFunc= fitnessFunc
Individual.vectorLength = 2

In [40]:
#Instantiate individual
individual = Individual()
print(individual)

--------------------------
[INFO]Float Vector
x_vec:
[4.776484205487502, -0.6068981844443888]
Fit:
39.354634865089864




# CrossOver

In [41]:
child = copy.deepcopy(individual)
child.vector_x = []
print(child)

--------------------------
[INFO]Float Vector
x_vec:
[]
Fit:
39.354634865089864




In [42]:
#Set alpha
alpha= random.uniform(0,1)
#Portion of each vector_x
l1_length = int(individual.vectorLength*alpha)
l2_length = individual.vectorLength - l1_length
#Portion of individual piece and other piece needed for crossOver
#Piece them together Creating new child
s1 = random.sample(individual.vector_x,l1_length)
s2 = random.sample(individual.vector_x,l2_length)
childVector = s1 + s2
child.vector_x = childVector

#ReEvaluate fitness
child.evaluateFitness()

print(child)

--------------------------
[INFO]Float Vector
x_vec:
[-0.6068981844443888, -0.6068981844443888]
Fit:
26.392432092374094




# Mutation

In [43]:
individual.sigma=individual.sigma*math.exp(individual.learningRate*individual.normprng.normalvariate(0,1))

if individual.sigma < individual.minSigma: individual.sigma=individual.minSigma
if individual.sigma > individual.maxSigma: individual.sigma=individual.maxSigma

for i,x in enumerate(individual.vector_x):
    #Shift the value of x to postive first
    tmp = x + abs(minLimit)
    #Bias can only be added, there would be no negative bias
    bias = (individual.maxLimit-individual.minLimit)*individual.sigma*individual.normprng.normalvariate(0,1)
    tmp = bias + tmp
    
    #Shift x back
    individual.vector_x[i] = tmp - abs(minLimit)

#ReEvaluate Fitness
individual.evaluateFitness()

print(individual)

--------------------------
[INFO]Float Vector
x_vec:
[1.2576000019243878, 1.5892105214171321]
Fit:
23.054243227500987




# Putting it all together

In [44]:
#Individuals(vector of floats)
#
# Individual.py
#
#

import math

#A simple 1-D Individual class
class Individual:
    """
    Individual
    """
    minSigma=1e-100
    maxSigma=1
    learningRate=1
    minLimit=None
    maxLimit=None
    uniprng=None
    normprng=None
    fitFunc=None
    vectorLength = None
    
    def __init__(self,name = None):
        self.vector_x = [self.uniprng.uniform(minLimit,maxLimit) for _ in range(self.vectorLength)]
        self.fit=self.__class__.fitFunc(self.vector_x)
        self.sigma=self.uniprng.uniform(0.9,0.1) #use "normalized" sigma
        self.name = name
        
    def crossover(self, other):
        child = copy.deepcopy(self)
        child.vector_x = []
        #Set alpha
        alpha= random.uniform(0,1)
        #Portion of each vector_x
        l1_length = int(self.vectorLength*alpha)
        l2_length = self.vectorLength - l1_length
        #Portion of self piece and other piece needed for crossOver
        #Piece them together Creating new child
        s1 = random.sample(self.vector_x,l1_length)
        s2 = random.sample(other.vector_x,l2_length)
        childVector = s1 + s2
        child.vector_x = childVector

        #ReEvaluate fitness
        child.evaluateFitness()
        
        return child
        
    
    def mutate(self):
        self.sigma=self.sigma*math.exp(self.learningRate*self.normprng.normalvariate(0,1))

        if self.sigma < self.minSigma: self.sigma=self.minSigma
        if self.sigma > self.maxSigma: self.sigma=self.maxSigma

        for i,x in enumerate(self.vector_x):
            if(self.sigma < 0.3): break #Simply not mutating
            #Shift the value of x to postive first
            tmp = x + abs(minLimit)
            #Bias can only be positive, there would be no negative bias
            bias = (self.maxLimit-self.minLimit)*self.sigma*self.normprng.normalvariate(0,1)
            tmp = bias + tmp

            #Shift x back
            self.vector_x[i] = tmp - abs(minLimit)

        #ReEvaluate Fitness
        self.evaluateFitness()
        
    def evaluateFitness(self):
        self.fit=self.__class__.fitFunc(self.vector_x)
        
    def __str__(self):
        print("--------------------------\n[INFO]Float Vector")
        if self.name is not None:
            print(f"Name :\n{self.name}")
        print(f"x_vec:\n{self.vector_x}")
        print(f"Fit:\n{self.fit}")
        return '\n'



In [45]:
#Set parameters
Individual.minSigma=1e-100
Individual.maxSigma=1
Individual.learningRate=1
Individual.minLimit=-5.12
Individual.maxLimit=5.12
Individual.uniprng= uniprng
Individual.normprng= normprng
Individual.fitFunc= fitnessFunc
Individual.vectorLength = 2

In [46]:
entity1 = Individual(name = 'entity1')
entity2 = Individual(name ='entity2')
print(entity1)
print(entity2)

--------------------------
[INFO]Float Vector
Name :
entity1
x_vec:
[4.208393855479032, 4.4981145330049435]
Fit:
55.35839435709722


--------------------------
[INFO]Float Vector
Name :
entity2
x_vec:
[1.7568100504369673, -4.26047255718826]
Fit:
31.467785870341803




In [55]:
child = entity1.crossover(entity2)
print(child)

--------------------------
[INFO]Float Vector
Name :
entity1
x_vec:
[4.4981145330049435, 1.7568100504369673]
Fit:
42.8909566558445




In [56]:
child.mutate()
print(child)

--------------------------
[INFO]Float Vector
Name :
entity1
x_vec:
[-3.2245502821364234, 5.938692147769848]
Fit:
44.80634113641544


